# Indexing Reward Unit Test
Performing verification of the indexing rewards mechanism by comparing the event data to the results of cadCAD model.

In [ ]:
import pandas as pd
import numpy as np
from decimal import Decimal
from decimal import *
import model.parts.utils as utils
from model.sys_params import *
df = pd.read_pickle(r'experiment.p')

Configure environment

In [ ]:
df.reset_index(inplace = True)
pd.set_option('display.max_rows', None)


In [ ]:

print("EXPECTED TRUE INDEXING REWARDS FROM REWARDS ASSIGNED EVENTS:")
is_first = True
for timestep, indexing_fee_events_list in rewards_assigned_events.items():
    # if is_first:
    #     is_first = False
    #     pass
    # else:        
        if indexing_fee_events_list is None:
            indexing_fee_amt = 0
        else:
            indexing_fee_amt = sum([e['amount'] for e in indexing_fee_events_list])        
        print(indexing_fee_amt)
        # print(f"{timestep}, {stake_deposited_event[0]['tokens']}")
    


In [ ]:
print("MODELED RESULTS")
# TODO: weave in indexer_revenue_cut events
# indexer #1
# indexer_revenue_cut = 0.89
# indexer #2
indexer_revenue_cut = Decimal(0.8)
# print(df.iloc[timestep])
# is_first = True
indexing_rewards_modeled = {}
for timestep, stake_deposited_event in rewards_assigned_events.items():
    # back indexing rewards out from increase in pool_delegated_stake / 0.11 * 0.89
    event = stake_deposited_event[0]
    new_indexing_rewards = df.iloc[timestep-1].indexers[event['indexer']].pool_delegated_stake
    old_indexing_rewards = df.iloc[timestep-2].indexers[event['indexer']].pool_delegated_stake
    indexing_rewards =(new_indexing_rewards - old_indexing_rewards) / (1 - indexer_revenue_cut) 
    indexing_rewards_modeled[timestep] = indexing_rewards
    print(f"{timestep}, {indexing_rewards=}")
        


In [ ]:
print("UNITTEST RESULTS")
is_first = True
# print(rewards_assigned_events)
cntExact = 0
cntReallyClose = 0
cntClose = 0
cnt = 0
cntWrong = 0
for timestep, rewards_assigned_event in rewards_assigned_events.items():
    # back indexing rewards out from increase in pool_delegated_stake / 0.11 * 0.89
    modeled_indexing_rewards = indexing_rewards_modeled[timestep]
    event_indexing_rewards = rewards_assigned_event[0]['amount']
    print(f"Event: Timestep={timestep}, Indexing Reward Tokens={event_indexing_rewards}")
    print(f"Model: Timestep={timestep}, Indexing Reward Tokens={modeled_indexing_rewards}")
    # print("Tokens Within 1%?", 0.99 <= abs(event_indexing_rewards / modeled_indexing_rewards) <= 1.01)
    # print()
    try:
        ratio = abs(modeled_indexing_rewards / event_indexing_rewards)
    except:
        ratio = np.inf
    exact = ratio == 1.0
    close = 0.99 <= ratio <= 1.01
    reallyClose = 0.99999 <= ratio <= 1.00001

    if exact:            
        cntExact += 1
    elif reallyClose:
        cntReallyClose += 1
    elif close:
        cntClose += 1
    else:
        cntWrong += 1
    cnt += 1
    print("Tokens Equal?", exact)
    print("Tokens Within 0.001%?", reallyClose)
    print("Tokens Within 1%?", close)
    print(f"Exact: {cntExact}, ReallyClose: {cntReallyClose}, Close: {cntClose}, Wrong: {cntWrong}, Total Number: {cnt}")   
    print()
print(f"Exact: {cntExact}, ReallyClose: {cntReallyClose}, Close: {cntClose}, Wrong: {cntWrong}, Total Number: {cnt}")           


# Conclusion
The indexing reward token quantities are accurate to roughly 21 significant figures except in timestep 58.  